<!-- docusaurus_head_meta::start
---
title: Custom Model Cost
---
docusaurus_head_meta::end -->

<!--- @wandbcode{prompt-optim-notebook} -->

# Setting up a custom cost model

Weave calculates costs based on the number of tokens used and the model used.
Weave grabs this usage and model from the output and associates them with the call.

Let's set up a simple custom model, that calculates its own token usage, and stores that in weave.

## Set up the environment

We install and import all needed packages.
We set `WANDB_API_KEY` in our env so that we may easily login with `wandb.login()` (this should be given to the colab as a secret).

We set the project in W&B we want to log this into in `name_of_wandb_project`.

**_NOTE:_** `name_of_wandb_project` may also be in the format of `{team_name}/{project_name}` to specify a team to log the traces into.

We then fetch a weave client by calling `weave.init()`

In [1]:
%pip install wandb weave datetime --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.1/374.1 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [3]:
import os

import wandb
from google.colab import userdata

import weave

os.environ["WANDB_API_KEY"] = userdata.get("WANDB_API_KEY")
name_of_wandb_project = "custom-cost-model"

wandb.login()

True

In [4]:
weave_client = weave.init(name_of_wandb_project)

Logged in as Weights & Biases user: jwlee64.
View Weave data at https://wandb.ai/jwlee64/custom-cost-model/weave


## Setting up a model with weave


In [5]:
from weave import Model


class YourModel(Model):
    attribute1: str
    attribute2: int

    def simple_token_count(self, text: str) -> int:
        return len(text) // 3

    # This is a custom op that we are defining
    # It takes in a string, and outputs a dict with the usage counts, model name, and the output
    @weave.op()
    def custom_model_generate(self, input_data: str) -> dict:
        # Model logic goes here
        # Here is where you would have a custom generate function
        prediction = self.attribute1 + " " + input_data

        # Usage counts
        prompt_tokens = self.simple_token_count(input_data)
        completion_tokens = self.simple_token_count(prediction)

        # We return a dictionary with the usage counts, model name, and the output
        # Weave will automatically associate this with the trace
        # This object {usage, model, output} matches the output of a OpenAI Call
        return {
            "usage": {
                "input_tokens": prompt_tokens,
                "output_tokens": completion_tokens,
                "total_tokens": prompt_tokens + completion_tokens,
            },
            "model": "your_model_name",
            "output": prediction,
        }

    # In our predict function we call our custom generate function, and return the output.
    @weave.op()
    def predict(self, input_data: str) -> dict:
        # Here is where you would do any post processing of the data
        outputs = self.custom_model_generate(input_data)
        return outputs["output"]

## Add a custom cost

Here we add a custom cost, and now that we have a custom cost, and our calls have usage, we can fetch the calls with `include_cost` and our calls with have costs under `summary.weave.costs`.

In [9]:
model = YourModel(attribute1="Hello", attribute2=1)
model.predict("world")

# We then add a custom cost to our project
weave_client.add_cost(
    llm_id="your_model_name", prompt_token_cost=0.1, completion_token_cost=0.2
)

# We can then query for the calls, and with include_costs=True
# we receive the costs back attached to the calls
calls = weave_client.get_calls(filter={"trace_roots_only": True}, include_costs=True)

list(calls)

🍩 https://wandb.ai/jwlee64/custom-cost-model/r/call/0191ed26-6b43-7901-8fd0-6c444794c1b2


[WeaveObject(Call(op_name='weave:///jwlee64/custom-cost-model/op/YourModel.predict:NpWYTb8xXYErwikUiWHzvgJPqXdOAOCQ816AXvr0OHI', trace_id='0191ed1e-cd04-7082-a0fc-55b1821ce2bb', project_id='jwlee64/custom-cost-model', parent_id=None, inputs={'self': ObjectRef(entity='jwlee64', project='custom-cost-model', name='YourModel', digest='ZoME9yTFGmehBF84YRElT076gwaGfGVKeYYzcEBEuEY', extra=()), 'input_data': 'world'}, id='0191ed1e-cd04-7082-a0fc-55a48d5b3229', output='Hello world', exception=None, summary={'usage': {'your_model_name': {'input_tokens': 1, 'output_tokens': 3, 'requests': 1}}, 'weave': {'status': <TraceStatus.SUCCESS: 'success'>, 'trace_name': 'YourModel.predict', 'latency_ms': 2, 'costs': {'your_model_name': {'prompt_tokens': 1, 'completion_tokens': 3, 'requests': 1, 'total_tokens': 0, 'prompt_tokens_total_cost': 0.10000000149011612, 'completion_tokens_total_cost': 0.6000000089406967, 'prompt_token_cost': 0.1, 'completion_token_cost': 0.2, 'prompt_token_cost_unit': 'USD', 'compl